In [3]:
import os
from dotenv import load_dotenv, find_dotenv

In [4]:
# make sure GOOGLE_API_KEY (or any other llm provider, must exactly the same)
_ = load_dotenv(find_dotenv())
# or put it into variable and pass it to llm model 
# google_api_key = os.getenv('GOOGLE_API_KEY') # pass this to the model creation

# Model Initiation

LangChain provides a class to support various LLM models, including OpenAI, Google Gemini, and Meta's LLaMA.
Initializing a model is simple—import the correct class and pass the model name and credentials (if not set in the environment).
For example, to use Google Gemini, import:

In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI 

/media/D/miniconda/envs/llm_intro/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


And then initialize the model with:

In [5]:
# available model in: https://cloud.google.com/vertex-ai/generative-ai/docs/learn/models
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

# Query the LLM

After initializing the model, you can query it using the `llm.invoke()` function with your prompt as a string argument.
This function returns a LangChain object with a content attribute that holds the model's response.

### Simple Invoke

In [7]:
response = llm.invoke("What is the good thing we can do to try LLM? Answer in 3 sentences.")

In [8]:
print(f"type(response) = {type(response)}")
print(response.content)

type(response) = <class 'langchain_core.messages.ai.AIMessage'>
Start with a simple, well-defined prompt to test its core capabilities.  Experiment with different phrasing of the same request to see how it handles variations.  Gradually increase complexity to explore its strengths and limitations in a controlled manner.



Apart from simple invocation, `llm.invoke()` supports three different input formats:

### Using PromptTemplate

A template allows you to create dynamic prompts by inserting variables into a predefined structure.
In LangChain, we use `PromptTemplate` for this purpose.

Similar to the llm object, a prompt template also has an `invoke()` function to process input and generate the final prompt.

In [13]:
from langchain_core.prompts import PromptTemplate
# Using template usually for passing arguments
template = """
What is the good thing we can do with LLM? Write in less than {max_words} words.
"""
prompt = PromptTemplate(template=template, input_variables=['max_words'])
llm.invoke(prompt.invoke({'max_words': 10}))

AIMessage(content='Automate tasks, generate creative content.\n', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-23f312bc-c5cc-4b78-9aff-1f6a37f44cc9-0', usage_metadata={'input_tokens': 24, 'output_tokens': 9, 'total_tokens': 33, 'input_token_details': {'cache_read': 0}})

LangChain overrides the pipe operator (|) to simplify chaining operations, making it easier to process prompts and model responses sequentially.

In [14]:
# or using chain
chain = prompt | llm # equal to llm.invoke(prompt.invoke({}))
chain.invoke({'max_words': 1}).content

'Innovate\n'

### Using Chat with Roles

LLM models also accept input in the form of chat messages.
In LangChain, each chat message is categorized into three distinct roles:

* System – Provides initial instructions (typically used only in the first message).
* Human – Represents user input or prompts.
* AI – Represents the LLM's response.

These roles align with OpenAI’s categories but use slightly different wording.

In [19]:
from langchain.schema import HumanMessage, SystemMessage, AIMessage

chats = (
    [
        SystemMessage(content="""
            You are a nice AI bot that helps a user figure out what to eat in one short sentence. 
            You can only respond to questions regarding food. 
            Any other questions that are asked not related to food will be answered with "I don't know" 
            """),
        HumanMessage(content="I like tomatoes, what should I eat?")
    ]
)
response = llm.invoke(chats)
print(response.content)

Try a caprese salad!



Trying unrelated question:

In [21]:
chats = (
    [
        SystemMessage(content="""
            You are a nice AI bot that helps a user figure out what to eat in one short sentence. 
            You can only respond to questions regarding food. 
            Any other questions that are asked not related to food will be answered with "I don't know" 
            """),
        HumanMessage(content="Who are the winner of 2024 world cup?")
    ]
)
response = llm.invoke(chats)
print(response.content)
print(type(response))

I don't know.

<class 'langchain_core.messages.ai.AIMessage'>


The return value of invoke() from the LLM is an AIMessage, which matches the chat format we've set up. This makes it easy to add the model's response to the ongoing chat history for future queries.

In [22]:
history = [
    SystemMessage(content="""
        You are a bot that will help a user to answer their question regarding AI. 
        Any other questions that are not part of AI, simply answer that you don't have the context for it
    """)
]

while True:
    next_sent = input(">> ")
    if next_sent == "break":
        break
    # convert the input into HummanMessage class, and append it to history
    history.append(HumanMessage(content=next_sent))
    
    # get responose
    response = llm.invoke(history)
    print(response.content)

    # add the response to the history 
    history.append(response)
    

>>  Who are the founder of Transformer architecture?


The Transformer architecture was primarily created by researchers at Google in 2017.  The key paper, "Attention is All You Need," lists Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, and Illia Polosukhin as authors.  While all contributed significantly, it's difficult to pinpoint a single "founder."  The paper represents a collaborative effort.



>>  Who have been to the moon?


I don't have the context to answer questions outside of artificial intelligence.



>>  break


In [23]:
# check for history
for h in history:
    print(h.type + " -- " + h.content[:100])

system -- 
        You are a bot that will help a user to answer their question regarding AI. 
        Any oth
human -- Who are the founder of Transformer architecture?
ai -- The Transformer architecture was primarily created by researchers at Google in 2017.  The key paper,
human -- Who have been to the moon?
ai -- I don't have the context to answer questions outside of artificial intelligence.



### Chat with Roles and Prompt

We can combine prompting with chat roles easily. 
Instead of manually creating instances of classes like SystemMessage, HumanMessage, and AIMessage, we can use a simpler input format: a pair of (role, content).

In [41]:
# see: https://python.langchain.com/docs/concepts/prompt_templates/
from langchain_core.prompts import ChatPromptTemplate

# the format is different, with system, user, and assistants category inside a tuple

MAX_WORD=50
LANGUAGE='Bahasa'

history = [
    # instead of creating a object, we pass pair (role, content)
    ("system", """
        You are a bot that will help a user to answer their question regarding AI. 
        Any other questions that are not part of AI, simply answer that you don't have the context for it.
        You only need to answer at most {max_word} words and use the language of {language}
    """
    ),
    ("user", "When the CNN is invented?")
]

# build prompt first
history_prompt = ChatPromptTemplate(history).invoke({'max_word': MAX_WORD, 'language': LANGUAGE})
response = llm.invoke(history_prompt) # or  

# or with chain operator
# response = (ChatPromptTemplate(history) | llm).invoke({'max_word': MAX_WORD, 'language': LANGUAGE})

print(response.content)

CNN, atau Convolutional Neural Network,  tidak memiliki penemu tunggal dan tanggal penemuan yang pasti. Konsepnya berkembang secara bertahap, dengan beberapa kontribusi penting di tahun 1980-an.



# Structured Output

Some llm models support returning in a specific and structured output.
This can be achieved by first creating the base class, with it's description and potential value, then call the invoke as usual.
What's difference is that the LLM will try to convert the answer into the class that we provided, thus making it easier to be used in the next process.
We use `pydantic` to create a class with type annotation and description.

See: https://python.langchain.com/docs/how_to/structured_output/

In [42]:
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field

In [43]:
class Country(BaseModel):
    name: str = Field(description='The country name')
    capital: str = Field(description='The capital country')
    income_per_capita_usd: int = Field(description='The income per capita in USD')

In [44]:
llm_structured = llm.with_structured_output(Country)

In [47]:
llm_structured.invoke("USA")

Country(name='USA', capital='Washington', income_per_capita_usd=65000)